In [ ]:
'''
Sources:
https://pyro4.readthedocs.io/en/stable/tutorials.html
https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory
https://stackoverflow.com/questions/13411544/delete-column-from-pandas-dataframe
https://stackoverflow.com/questions/33742588/pandas-split-dataframe-by-column-value
https://stackoverflow.com/questions/45759966/counting-unique-values-in-a-column-in-pandas-dataframe-like-in-qlik/45760042
https://chrisalbon.com/python/data_wrangling/pandas_list_unique_values_in_column/
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html
https://www.guru99.com/python-check-if-file-exists.html
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
'''

In [ ]:
from DataAccessor import DataAccessor
from Pyro5.api import locate_ns
from Pyro5.client import Proxy
from Pyro5.errors import excepthook as pyro5_excepthook
import sys
import threading
from concurrent.futures import ThreadPoolExecutor
import multiprocessing as mp
import socket

In [ ]:
def find_datawarehouse():
    nameserver = locate_ns()
    datawarehouse_uri = nameserver.lookup("example.datawarehouse")
    return Proxy(datawarehouse_uri)

if __name__ ==  '__main__':
    sys.excepthook = pyro5_excepthook
    
    datawarehouse = None
    connectionRefused = True
    while(connectionRefused):
        try:
            datawarehouse = find_datawarehouse()
            connectionRefused = False
        except:
            connectionRefused = True
    
    dataaccessor = DataAccessor(datawarehouse)
    
    workers = [str(socket.gethostname()) + "_" + str(i + 1) for i in range(0, mp.cpu_count())]
    
    pool = mp.Pool(mp.cpu_count())
    execSummary = pool.map(dataaccessor.performDataCleansing, workers)
    pool.close()
    
    print(str(execSummary))